## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate  source  \
0  2025-10-14-10-52-43 +0000  nypost   
1  2025-10-14-10-49-39 +0000     bbc   
2  2025-10-14-10-48-48 +0000     nyt   
3  2025-10-14-10-48-44 +0000    wapo   
4  2025-10-14-10-47-57 +0000     nyt   

                                               title  \
0  President Trump accuses Time magazine of delib...   
1  Personal data potentially stolen in Asahi cybe...   
2  Slowing Electric Vehicle Sales Will Cost G.M. ...   
3  Madagascar’s president dissolves National Asse...   
4  Ukraine’s Formula for Peace: Fewer Talks, More...   

                                                link  
0  https://nypost.com/2025/10/14/us-news/presiden...  
1  https://www.bbc.com/news/articles/c0rpwk51qxro...  
2  https://www.nytimes.com/2025/10/14/business/ge...  
3  https://www.washingtonpost.com/world/2025/10/1...  
4  https://www.nytimes.com/2025/10/14/world/europ...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
1        trump     56
295       gaza     21
249     israel     18
0    president     18
43       china     17

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
198  2025-10-13-20-53-37 +0000    wapo   
242  2025-10-13-17-58-10 +0000  nypost   
229  2025-10-13-18-40-17 +0000  nypost   
0    2025-10-14-10-52-43 +0000  nypost   
195  2025-10-13-20-57-45 +0000     bbc   

                                                 title  \
198  Trump and world leaders meet in Egypt on futur...   
242  Trump boasts Gaza peace process could be presi...   
229  Trump trolls Norway rep over Nobel Peace Prize...   
0    President Trump accuses Time magazine of delib...   
195  Israeli hostages and Palestinian detainees fre...   

                                                  link  score  
198  https://www.washingtonpost.com/world/2025/10/1...    136  
242  https://nypost.com/2025/10/13/us-news/trump-sa...    133  
229  https://nypost.com/2025/10/13/us-news/trump-tr...    127  
0    https://nypost.com/2025/10/14/us-news/presiden...    124  
195  https://www.bbc.com/news/articles/c740jx07vz0o...    122

## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate  source  \
198    136  2025-10-13-20-53-37 +0000    wapo   
0       68  2025-10-14-10-52-43 +0000  nypost   
223     52  2025-10-13-19-08-18 +0000  nypost   
113     51  2025-10-14-02-41-46 +0000  nypost   
305     50  2025-10-13-13-11-27 +0000  nypost   
118     50  2025-10-14-01-59-18 +0000     cbc   
126     47  2025-10-14-01-00-00 +0000     wsj   
85      45  2025-10-14-05-40-36 +0000  nypost   
195     38  2025-10-13-20-57-45 +0000     bbc   
170     34  2025-10-13-21-59-00 +0000     wsj   

                                                 title  \
198  Trump and world leaders meet in Egypt on futur...   
0    President Trump accuses Time magazine of delib...   
223  Israeli hostages returning to their families l...   
113  Ex-student allegedly stabs 3 staff members, pl...   
305  Dow soars nearly 600 points after Trump, Vance...   
118  New York Times, AP, Reuters and Newsmax among ...   
126  The White House move to lay off roughly 4,000 ...   
85   82-year-old woman shot husband dead at South C...   
195  Israeli hostages and Palestinian detainees fre...   
170  What has Trump done as president so far? The W...   

                                                  link  
198  https://www.washingtonpost.com/world/2025/10/1...  
0    https://nypost.com/2025/10/14/us-news/presiden...  
223  https://nypost.com/2025/10/13/world-news/israe...  
113  https://nypost.com/2025/10/13/us-news/ex-stude...  
305  https://nypost.com/2025/10/13/business/dow-fut...  
118  https://www.cbc.ca/news/world/new-york-times-a...  
126  https://www.wsj.com/politics/policy/trump-gove...  
85   https://nypost.com/2025/10/14/us-news/82-year-...  
195  https://www.bbc.com/news/articles/c740jx07vz0o...  
170  https://www.wsj.com/politics/policy/donald-tru...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
